# Problemas en el FRONTEND

La arquitectura construida hasta ahora son dos servicios, uno para la BD y otra la API, que se levantan con un `docker compose`, y un contenedor de **nginx** como proxy inverso que se levanta con `docker run`:

```sh
docker run --rm --name my-nginx-container -p 80:80 \
  --network platzinet \
  -v /home/carlos/Documents/tutienda-campesina-app/web-page:/usr/share/nginx/html:ro \
  my-nginx-proxy```

Sin embargo se noto, y se desperdicio tiempo valioso, que no se estaban cargando los archivos estaticos. Un simple cambio de por ejemplo el color de un titulo, no los tomaba. Y por tanto todos los intentos por hacer un FRONTEND, medianamente basico FRACASARON. 💀 😠

## Primera SOlucion  

Despues de horas y horas, lo unico que se me ocurrio fue copiar directamente los estilos en `index.html`, y no creeras, funciono. Pero busque otra solucion. 

![image](https://imgur.com/aUKUkHF.png)

## SOLUCION DEFINITIVA

La solucion en mi maquina LINUX, fue usar otro PROXY INVERSO el clasico servidor `APACHE`, que tiene una configuracion especial, donde en el directorio `/etc/apache2/sites-available/`, guardaremos el archivo `tutienda-proxy.conf`, del cual hay una copia en este proyecto:

`sudo  cat /etc/nginx/sites-available/tutienda-proxy.conf`

### Comandos utiles de APACHE:

- `sudo apache2ctl configtest`: Prueba la configuracion de Apache
- `sudo systemctl restart apache2`: Reiniciar Apache.

Y Mira que bonito hace una consulta:

![image](https://imgur.com/mkap26M.png)

Y si hay un error en la busqueda:

![image](https://imgur.com/ulK8oco.png)


Todo lo anterior, se desarrollo usando logica de JS desde el archivo `my-scrip.js`, usando clases y metodos de `JQUERY`, para encontrar elementos en el HTML, mostrarlos y ocultarlos segun sea el caso. Por ejemplo:

```js
        .then(function(client){
            console.log(`cliente con id:${client.id} encontrado`);
            $('#result-container').show(); 👈
            $('.message').show();
            $('#error-message').hide();
//            $('#clientForm').hide();
            $('#initial-message').hide();


            $('#client-id-display').html(client.id); 👈
            $('#client-name-display').html(client.name);
            $('#client-lastname-display').html(client.lastName);
            $('#client-dob-display').html(client.DOB);            

        })
```

### mostro la pagina por defecto de apache


docker run -d --name my-apache-server --network platzinet -p 80:80 ubuntu/apache2:2.4.62

docker exec -it my-apache-server bash

### mostro mi propia html, css, con estilos. 

BIND MOUNT, hot reload:

docker run -d \                                                                            ok
  --name my-apache-server \
  --network platzinet \
  -p 80:80 \
  -v /home/julian/documents/tutienda-campesina-app/web-page:/var/www/html \
  ubuntu/apache2:2.4.62

si muestra los estilos, todavia sin peticiones a la API


### 


docker run -d \                                                                    
  --name my-apache-server \
  --network platzinet \
  -p 80:80 \
  -v /home/julian/documents/tutienda-campesina-app/web-page:/var/www/html \
  -v /home/julian/documents/tutienda-campesina-app/apache-config/my-apache-proxy.conf:/etc/apache2/sites-available/000-default.conf \
  ubuntu/apache2:2.4.62 /bin/bash -c "a2enmod proxy proxy_http && apache2ctl -D FOREGROUND"


ESTE FUNCIONA:

docker run -d --name my-apache-server --network platzinet -p 80:80 -v /home/julian/documents/tutienda-campesina-app/web-page:/var/www/html -v /home/julian/documents/tutienda-campesina-app/apache-config/my-apache-proxy.conf:/etc/apache2/sites-available/000-default.conf ubuntu/apache2:2.4.62 /bin/bash -c "a2enmod proxy proxy_http && apache2ctl -D FOREGROUND"



